In [1]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from numpy.linalg import matrix_power
from scipy.linalg import expm, expm_frechet
import scipy

In [2]:
def PauliMatrix(i):
    if i == 0:
        return np.array([[1, 0], [0, 1]])
    elif i == 1:
        return np.array([[0, 1], [1, 0]])
    elif i == 2:
        return np.array([[0, -1j], [1j, 0]])
    elif i == 3:
        return np.array([[1, 0], [0, -1]])
    else:
        raise ValueError("i must be 0, 1, 2, or 3.")

def PauliTensor(i, j):
    return np.kron(PauliMatrix(i), PauliMatrix(j))

In [19]:
def xgate_model(detuning):
    """
    xstate = (Delta, Omega, phi)
    """
    return expm(-(1j/2)*
                (detuning*PauliMatrix(3) + (np.pi/2)*PauliMatrix(1))
            )

def ygate_model(detuning):
    """
    xstate = (Delta, Omega, phi)
    """
    return expm(-(1j/2)*
                (detuning*PauliMatrix(3) + (np.pi/2)*PauliMatrix(2))
            )

def multiply_sequence(sequence, Xdef, Ydef):
    """Multiply a sequence of X and Y gates together."""
    circ = np.eye(2)
    for c in sequence:
        if c == 'X':
            circ = Xdef@circ
        elif c == 'Y':
            circ = Ydef@circ
        else:
            raise ValueError("sequence must only contain 'X' and 'Y' characters.")
    return circ

def probability(detuning, d, circ_def):
    # assume a convention where the gates go from left to right in the string
    Xgate = xgate_model(detuning)
    Ygate = ygate_model(detuning)
    germ, prep, meas = circ_def
    prep_unitary = multiply_sequence(prep, Xgate, Ygate)
    germ_unitary = multiply_sequence(germ, Xgate, Ygate)
    meas_unitary = multiply_sequence(meas, Xgate, Ygate)
    circ_unitary = meas_unitary@np.linalg.matrix_power(germ_unitary, d)@prep_unitary
    prob = abs(circ_unitary[0, 0])**2
    return prob

def probability_grad(detuning, d, circ_def, epsilon=1e-6):
    # calculate with finite difference
    if type(detuning) == list:
        detuning = detuning[0]
    p0 = probability(detuning-epsilon, d, circ_def)
    p1 = probability(detuning+epsilon, d, circ_def)
    return (p1-p0)/(2*epsilon)
    

In [20]:
def edesign_probability(detuning, edesign):
    probs = dict()
    for idx, d in enumerate(edesign.keys()):
        probs[d] = []
        for circ_def in edesign[d]:
            probs[d].append( probability(detuning, d, circ_def) )
    return probs

def edesign_gradients(detuning, edesign):
    prob_grads = dict()
    for idx, d in enumerate(edesign.keys()):
        prob_grads[d] = []
        for circ_def in edesign[d]:
            prob_grads[d].append( probability_grad(detuning, d, circ_def) )
    return prob_grads

def edesign_observation(detuning, edesign, num_shots=1000):
    probs = edesign_probability(detuning, edesign)
    emperical_dists = dict()
    for d in edesign.keys():
        emperical_dists[d] = []
        for prob in probs[d]:
            emperical_dists[d].append(np.random.binomial(num_shots, prob))
    return emperical_dists

In [21]:
def fisher_information_matrix(xstate, edesign):
    ed_probs = edesign_probability(xstate, edesign)
    ed_grads = edesign_gradients(xstate, edesign)
    probs = [value for sublist in ed_probs.values() for value in sublist]
    grads = [value for sublist in ed_grads.values() for value in sublist]
    fisher_info = np.zeros((len(xstate), len(xstate)))
    for idx, p in enumerate(probs):
        if p == 0:
            fisher_info += np.zeros((len(xstate), len(xstate)))
        else:
            fisher_info += np.outer(grads[idx], grads[idx])/p
    return fisher_info

def score_fisher_information_matrix(fmat, clip=1e-19):
    evals = np.linalg.eigvals(fmat)
    # clip 
    evals = np.clip(evals, clip, float('inf'))
    if np.any(evals <= 0):
        print(evals, np.clip(evals, clip, np.max(evals)))
    return np.sum(np.log(evals))

In [40]:
def find_minimial_circuits_fixed_depth(xestimate, d, num_circs, germ_list, prep_list, meas_list):
    """Find the minimal circuits for a fixed depth."""
    all_circuits = dict()
    for germ in germ_list:
        for prep in prep_list:
            for meas in meas_list:
                all_circuits[(germ, prep, meas)] = prep + germ*d + meas
    
    minimal_circuits = []
    total_fisher_info = np.zeros((len(xestimate), len(xestimate)))
    for r in range(num_circs):
        scores = {}
        for circ_def, circ in all_circuits.items():
            if circ_def in minimal_circuits:
                continue
            I_circ = fisher_information_matrix(xestimate, {d: [circ_def]})
            new_fisher = total_fisher_info + I_circ
            score = score_fisher_information_matrix(new_fisher) 
            scores[circ_def] = score
        best_circ_def = max(scores, key=scores.get)
        minimal_circuits.append(best_circ_def)
        total_fisher_info += fisher_information_matrix(xestimate, {d: [best_circ_def]})
    if np.linalg.matrix_rank(total_fisher_info) < len(xestimate):
        Warning( np.linalg.matrix_rank(total_fisher_info) )
    return minimal_circuits

In [38]:
def list_best_experiments(xestimate, d, germ_list, prep_list, meas_list, num_best=5):
    all_circuits = dict()
    for germ in germ_list:
        for prep in prep_list:
            for meas in meas_list:
                all_circuits[(germ, prep, meas)] = prep + germ*d + meas

    # return num_best best experiments
    scored_circuits = dict()
    for circ_def in all_circuits.keys():
        scored_circuits[circ_def] = score_fisher_information_matrix(fisher_information_matrix(xestimate, {d: [circ_def]}))
    return sorted(scored_circuits.items(), key=lambda x: x[1], reverse=True)[:num_best]
    

In [39]:
all_state_preps = ['', 'Y', 'YY', 'YYY', 'X', 'XXX']
all_measurements = ['', 'Y', 'YY', 'YYY', 'X', 'XXX']
germ_list = ['X', 'Y', 'XX', 'YY', 'XY', 'XXY', 'XYX', 'XXYY', 'YYXXXYYXXXXYY']

In [42]:
depths = [1, 2, 3, 4]
for d in depths:
    print(list_best_experiments([0], d, germ_list, all_state_preps, all_measurements))
    print(find_minimial_circuits_fixed_depth([0], d, 5, germ_list, all_state_preps, all_measurements))

[(('XXYY', 'YYY', 'Y'), 70.60877719744526), (('XYX', 'X', 'YYY'), 16.78458418690677), (('XYX', 'YYY', 'X'), 16.52198908860993), (('XY', 'YY', 'XXX'), 16.448752243420316), (('XXYY', 'XXX', 'X'), 15.95923354606351)]
[('XXYY', 'YYY', 'Y'), ('X', '', ''), ('X', '', 'Y'), ('X', '', 'YY'), ('X', '', 'YYY')]
[(('XXYY', 'Y', 'Y'), 16.888357036658824), (('XXYY', 'X', 'X'), 16.888357036658824), (('XY', 'X', 'YY'), 16.207682276167596), (('XXY', '', 'YY'), 15.968327100998309), (('XXY', 'Y', 'Y'), 15.968327100998309)]
[('XXYY', 'Y', 'Y'), ('XXYY', 'X', 'X'), ('XY', 'X', 'YY'), ('XXY', '', 'YY'), ('XXY', 'Y', 'Y')]
[(('XXYY', 'YYY', 'Y'), 67.71493923157261), (('XYX', 'X', 'YYY'), 17.10678057251156), (('XYX', 'YYY', 'X'), 16.944223969776804), (('XYX', 'Y', 'XXX'), 16.253840522056155), (('XYX', 'XXX', 'Y'), 16.09137930046001)]
[('XXYY', 'YYY', 'Y'), ('X', '', ''), ('X', '', 'Y'), ('X', '', 'YY'), ('X', '', 'YYY')]
[(('XXYY', 'Y', 'Y'), 18.27465139794993), (('XXYY', 'X', 'X'), 18.27465139794993), (('XY

In [44]:
depths = [8, 16, 32, 64]
for d in depths:
    print(find_minimial_circuits_fixed_depth([0], d, 8, germ_list, all_state_preps, all_measurements))

[('XXYY', 'Y', 'Y'), ('XXYY', 'X', 'X'), ('XXYY', 'YYY', 'YYY'), ('XXYY', 'XXX', 'XXX'), ('XY', 'X', 'YY'), ('XXY', 'YY', ''), ('XXY', '', 'YY'), ('XXY', 'Y', 'Y')]
[('XXYY', 'Y', 'Y'), ('XXYY', 'X', 'X'), ('XXYY', 'YYY', 'YYY'), ('XXYY', 'XXX', 'XXX'), ('XY', 'YY', 'XXX'), ('XXY', 'Y', 'Y'), ('XXY', 'YY', ''), ('XXY', '', 'YY')]
[('XXYY', 'Y', 'Y'), ('XXYY', 'X', 'X'), ('XXYY', 'YYY', 'YYY'), ('XXYY', 'XXX', 'XXX'), ('XY', 'X', 'YY'), ('XXY', 'Y', 'Y'), ('XXY', 'YY', ''), ('XXY', '', 'YY')]
[('XXYY', 'Y', 'Y'), ('XXYY', 'X', 'X'), ('XXYY', 'YYY', 'YYY'), ('XXYY', 'XXX', 'XXX'), ('XY', 'YY', 'XXX'), ('XXY', 'Y', 'Y'), ('XXY', 'YY', ''), ('XXY', '', 'YY')]
